### Conduct exeriment

In [ ]:
import numpy as np
import time
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.extensions import Initialize
from qiskit_textbook.tools import random_state, array_to_latex

from qiskitflow import Experiment

def create_bell_pair(qc, a, b):
    """Creates a bell pair in qc using qubits a & b"""
    qc.h(a) # Put qubit a into state |+>
    qc.cx(a,b) # CNOT with a as control and b as target

    
def alice_gates(qc, psi, a):
    qc.cx(psi, a)
    qc.h(psi)

    
def bob_gates(qc, qubit, crz, crx):
    # Here we use c_if to control our gates with a classical
    # bit instead of a qubit
    qc.x(qubit).c_if(crx, 1) # Apply gates if the registers 
    qc.z(qubit).c_if(crz, 1) # are in the state '1'
    
    
def measure_and_send(qc, a, b):
    """Measures qubits a & b and 'sends' the results to Bob"""
    qc.barrier()
    qc.measure(a,0)
    qc.measure(b,1)



with Experiment("quantum teleportation") as experiment:
    start_time = time.time()
    
    # conduct experiment as usual
    psi = random_state(1)
    init_gate = Initialize(psi)
    init_gate.label = "init"
    inverse_init_gate = init_gate.gates_to_uncompute()

    qr = QuantumRegister(3, name="q")   # Protocol uses 3 qubits
    crz = ClassicalRegister(1, name="crz") # and 2 classical registers
    crx = ClassicalRegister(1, name="crx")
    qc = QuantumCircuit(qr, crz, crx)
    qc.append(init_gate, [0])
    qc.barrier()
    create_bell_pair(qc, 1, 2)
    qc.barrier()
    alice_gates(qc, 0, 1)
    measure_and_send(qc, 0, 1)
    bob_gates(qc, 2, crz, crx)
    qc.append(inverse_init_gate, [2])
    cr_result = ClassicalRegister(1)
    qc.add_register(cr_result)
    qc.measure(2,2)
    backend_name = "qasm_simulator"
    backend = BasicAer.get_backend(backend_name)
    counts = execute(qc, backend, shots=1024).result().get_counts()
    
    end_time = time.time()
    
    runtime = end_time - start_time
    
    # qiskitflow
    # log parameters used
    experiment.write_parameter("backend name", backend_name)
    # log metrics of experiment
    experiment.write_metric("runtime", runtime)
    # log counts of experiment
    experiment.write_counts("experiment counts", counts)

### Review results

In [2]:
! qiskitflow runs


QiskitFlow. Reproducible quantum experiments.

Experiment runs

+---------------------+------------------------------------+----------------------------+-----------------------------+
| Date                | Name                               | Metrics                    | Parameters                  |
+=====================+====================================+============================+=============================+
| 2021-03-06 20:51:33 | quantum teleportation              | runtime:0.3859751224517822 | backend name:qasm_simulator |
|                     | (f530f0c61b9542b0b13a02dd9133abd5) |                            |                             |
+---------------------+------------------------------------+----------------------------+-----------------------------+


In [4]:
! qiskitflow run f530f0c61b9542b0b13a02dd9133abd5


QiskitFlow. Reproducible quantum experiments.

Run [f530f0c61b9542b0b13a02dd9133abd5] detailed information.

+------------+----------------------------------+
|            | Values                           |
+============+==================================+
| Run ID     | f530f0c61b9542b0b13a02dd9133abd5 |
+------------+----------------------------------+
| Experiment | quantum teleportation            |
+------------+----------------------------------+
| Date       | 2021-03-06 20:51:33              |
+------------+----------------------------------+
| Parameters | backend name:qasm_simulator      |
+------------+----------------------------------+
| Metrics    | runtime:0.3859751224517822       |
+------------+----------------------------------+
| Counts     | experiment counts:               |
|            | - 0 1 1 -> 258                   |
|            | - 0 1 0 -> 236                   |
|            | - 0 0 0 -> 272                   |
|            | - 0 0 1 -> 258           

In [5]:
! qiskitflow share f530f0c61b9542b0b13a02dd9133abd5 --user=admin --password=admin


QiskitFlow. Reproducible quantum experiments.

Sharing run [f530f0c61b9542b0b13a02dd9133abd5]
Run was successfully shared!
